In [1]:
# Here the libraries that are needed for the experiment are imported
import subprocess
import json
import pandas as pd
import textstat
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import datasets
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch

In [2]:
# Loading the MMLU dataset
mmlu_dataset = datasets.load_dataset("cais/mmlu", "all", split="dev")

# Extraction of unique topics
mmlu_topics = list(set(mmlu_dataset["subject"]))

# Parsing of names
mmlu_topics = [topic.replace("_", " ") for topic in mmlu_topics]

print(f"Total topics: {len(mmlu_topics)}")

# First ten
print(mmlu_topics)

Total topics: 57
['high school statistics', 'college physics', 'human sexuality', 'international law', 'logical fallacies', 'business ethics', 'high school chemistry', 'high school government and politics', 'high school macroeconomics', 'moral disputes', 'public relations', 'abstract algebra', 'computer security', 'high school european history', 'miscellaneous', 'world religions', 'clinical knowledge', 'high school psychology', 'marketing', 'security studies', 'astronomy', 'formal logic', 'philosophy', 'professional law', 'high school mathematics', 'prehistory', 'nutrition', 'professional medicine', 'econometrics', 'college biology', 'high school geography', 'professional accounting', 'elementary mathematics', 'high school world history', 'global facts', 'jurisprudence', 'medical genetics', 'human aging', 'high school biology', 'virology', 'high school us history', 'college mathematics', 'anatomy', 'college medicine', 'high school computer science', 'moral scenarios', 'sociology', 'con

In [3]:
# First 5 topics are predifined for the experiment
TOPICS = ['human aging', 'computer security', 'astronomy', 'prehistory', 'nutrition']
TOPICS

['human aging', 'computer security', 'astronomy', 'prehistory', 'nutrition']

In [4]:
# CLEAR corpus is used to get texts as an examples for certain readability levels. It will be
# use for Few-Shot technique, where I need examples of text of defined readability level.
df = pd.read_csv("CLEAR.csv")
df.columns

Index(['ID', 'Last Changed', 'Author', 'Title', 'Anthology', 'URL', 'Source',
       'Pub Year', 'Category', 'Location', 'License', 'MPAA\nMax',
       'MPAA \n#Max', 'MPAA\n#Avg', 'Excerpt', 'Google\nWC', 'Joon\nWC v1',
       'British WC', 'British Words', 'Sentence\nCount v1',
       'Sentence\nCount v2', 'Paragraphs', 'BT Easiness', 'BT s.e.',
       'Flesch-Reading-Ease', 'Flesch-Kincaid-Grade-Level',
       'Automated Readability Index', 'SMOG Readability',
       'New Dale-Chall Readability Formula', 'CAREC', 'CAREC_M', 'CARES',
       'CML2RI', 'firstPlace_pred', 'secondPlace_pred', 'thirdPlace_pred',
       'fourthPlace_pred', 'fifthPlace_pred', 'sixthPlace_pred',
       'Kaggle split'],
      dtype='object')

In [5]:
# Here I search for texts based on their Flesch-Kincaid grade level, as most universal.
readability_column = "Flesch-Kincaid-Grade-Level"

# Function to map readability level
def map_readability_level(score):
    try:
        score = float(score)
        if score < 4:
            return "Early Elementary"
        elif score < 7:
            return "Upper Elementary"
        elif score < 9:
            return "Middle School"
        elif score < 13:
            return "High School"
        elif score < 15:
            return "College"
        elif score < 17:
            return "Advanced College"
        elif score < 18:
            return "Graduate"
        else:
            return "Research"
    except:
        return "Unknown"

# Apply readability mapping
df["Readability Level"] = df[readability_column].apply(map_readability_level)

# Filter out rows without level or text
df_filtered = df[(df["Readability Level"] != "Unknown") & (df["Excerpt"].notna())]

# I get two samples for readability level, it can be more, I can use it as a parameter
excerpt_samples = (
    df_filtered
    .groupby("Readability Level")
    .apply(lambda g: g.sample(n=min(2, len(g)), random_state=42)[["Excerpt", "Title", "URL"]])
    .reset_index(level=0)
    .rename(columns={"level_0": "Readability Level"})
)

# Store result
excerpt_df = excerpt_samples.copy()

/var/folders/36/5j3ncst55bz7ldyyc01k3kwh0000gn/T/ipykernel_51433/2444068984.py:37: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.sample(n=min(2, len(g)), random_state=42)[["Excerpt", "Title", "URL"]])


In [6]:
excerpt_df

,Readability Level,Excerpt,Title,URL
3491,Advanced College,It is hardly possible to imagine with what eag...,Life of Robert Burns,http://www.online-literature.com/thomas-carlyl...
463,Advanced College,Debugging is the process of finding and resolv...,Debugging,https://en.wikipedia.org/wiki/Debugging
4671,College,"Armed neutrality, in international politics, i...",Armed_neutrality,https://en.wikipedia.org/wiki/Armed_neutrality
771,College,Protective tariffs are tariffs that are enacte...,Protective_tariff,https://en.wikipedia.org/wiki/Protective_tariff
4708,Early Elementary,The Sun’s light shines on Earth. The light mak...,Getting Warmer,http://static.ehe.osu.edu/sites/beyond/penguin...
1327,Early Elementary,Cat and Dog walk. They walk in their village. ...,Cat and Dog\nand the egg,https://www.africanstorybook.org/#
11,Graduate,It was believed by the principal men of Virgin...,For Love of Country A Story of Land and Sea in...,http://www.gutenberg.org/cache/epub/20791/pg20...
450,Graduate,"Cosmology (from the Greek kosmos, ""world"" and ...",Cosmology,https://en.wikipedia.org/wiki/Cosmology
1621,High School,There is no specific scientific calculator pre...,Technical Mechanics Grade 10,http://ukuqonda.co.za/digicom/Grade-10-Technic...
2880,High School,"Daniel was but eight years old, a boy of strik...",From Farm Boy to Senator,http://www.gutenberg.org/files/53382/53382-h/5...


In [7]:
# Checks for text length, done because I though that they can be too big.
excerpt_df["Char Length"] = excerpt_df["Excerpt"].str.len()
excerpt_df["Word Count"] = excerpt_df["Excerpt"].str.split().apply(len)

# Find min and max
min_char = excerpt_df["Char Length"].min()
max_char = excerpt_df["Char Length"].max()
min_word = excerpt_df["Word Count"].min()
max_word = excerpt_df["Word Count"].max()

print(f"[excerpt_df] Excerpt Length (Characters): Min = {min_char}, Max = {max_char}")
print(f"[excerpt_df] Excerpt Length (Words): Min = {min_word}, Max = {max_word}")

[excerpt_df] Excerpt Length (Characters): Min = 747, Max = 1199
[excerpt_df] Excerpt Length (Words): Min = 148, Max = 198


In [8]:
# For future work of result analysis, accuracy analysis. Mapping of metrics results and levels
# I will group them, but also try to differentiate for different grade levels, to compare.
# Also some metrics are not universal, and more adapted for school grade level, like SPACHE
# this is interesting to analyze. Universal mapping helps to compare

FLESCH_KINCAID_GRADE = {
    1: "Early Elementary",
    2: "Early Elementary",
    3: "Early Elementary",
    4: "Upper Elementary",
    5: "Upper Elementary",
    6: "Upper Elementary",
    7: "Middle School",
    8: "Middle School",
    9: "High School",
    10: "High School",
    11: "High School",
    12: "High School",
    13: "College",
    14: "College",
    15: "Advanced College",
    16: "Advanced College",
    17: "Graduate",
    18: "Research"
}

FLESCH_READING_EASE = {
    (90, 100): "Upper Elementary",
    (80, 90): "Upper Elementary",
    (70, 80): "Middle School",
    (65, 70): "Middle School",
    (60, 65): "High School",
    (50, 60): "High School",
    (40, 50): "High School",
    (30, 40): "High School",
    (0, 30): "College"
}

DALE_CHALL = {
    (0.0, 4.5): "Upper Elementary",
    (4.5, 5.0): "Upper Elementary",
    (5.0, 5.5): "Upper Elementary",
    (5.5, 6.0): "Upper Elementary",
    (6.0, 6.5): "Middle School",
    (6.5, 7.0): "Middle School",
    (7.0, 7.5): "High School",
    (7.5, 8.0): "High School",
    (8.0, 8.5): "High School",
    (8.5, 9.0): "College",
    (9.0, 10.0): "College",
    (10.0, float("inf")): "Graduate"
}

ARI = {
    1: "Early Elementary",
    2: "Early Elementary",
    3: "Early Elementary",
    4: "Upper Elementary",
    5: "Upper Elementary",
    6: "Upper Elementary",
    7: "Middle School",
    8: "Middle School",
    9: "High School",
    10: "High School",
    11: "High School",
    12: "High School",
    13: "College",
    14: "College",
    15: "Advanced College",
    16: "Advanced College",
    17: "Graduate",
    18: "Research",
    19: "Research",
    20: "Legal / Medical"
}

COLEMAN_LIAU = {
    1: "Early Elementary",
    2: "Early Elementary",
    3: "Early Elementary",
    4: "Upper Elementary",
    5: "Upper Elementary",
    6: "Upper Elementary",
    7: "Middle School",
    8: "Middle School",
    9: "High School",
    10: "High School",
    11: "High School",
    12: "High School",
    13: "College",
    14: "College",
    15: "Advanced College",
    16: "Advanced College",
    17: "Graduate",
    18: "Research"
}

GUNNING_FOG = {
    6: "Upper Elementary",
    7: "Middle School",
    8: "Middle School",
    9: "High School",
    10: "High School",
    11: "High School",
    12: "High School",
    13: "College",
    14: "College",
    15: "Advanced College",
    16: "Advanced College",
    17: "Graduate",
    18: "Research",
    19: "Research",
    20: "Legal / Medical"
}

SMOG_INDEX = {
    6: "Upper Elementary",
    7: "Middle School",
    8: "Middle School",
    9: "High School",
    10: "High School",
    11: "High School",
    12: "High School",
    13: "College",
    14: "College",
    15: "Advanced College",
    16: "Advanced College",
    17: "Graduate",
    18: "Research",
    19: "Legal / Medical"
}

SPACHE = {
    1: "Early Elementary",
    2: "Early Elementary",
    3: "Early Elementary",
    4: "Upper Elementary"
}

LINSEAR_WRITE = {
    4: "Upper Elementary",
    5: "Upper Elementary",
    6: "Upper Elementary",
    7: "Middle School",
    8: "Middle School",
    9: "High School",
    10: "High School",
    11: "High School",
    12: "High School",
    13: "College",
    14: "College",
    15: "Advanced College",
    16: "Graduate"
}

LEXILE = {
    200: "Early Elementary",
    300: "Early Elementary",
    400: "Early Elementary",
    500: "Upper Elementary",
    600: "Upper Elementary",
    700: "Upper Elementary",
    800: "Middle School",
    900: "Middle School",
    1000: "High School",
    1100: "High School",
    1200: "High School",
    1300: "High School",
    1400: "College",
    1500: "Advanced College",
    1600: "Graduate",
    1700: "Research",
    1800: "Legal / Medical"
}


In [9]:
# It was proposed to have specific text lengths to iterate
OUTPUT_LENGTHS = [50, 100, 150, 200]

In [10]:
# I started with two models, Mistral and Llama. One has 7B parameters, another 8B. They are close
# that is why it is possible to compare them.
model_mistral = "mistral:instruct"
model_llama = "llama3.1:8b"

def generate_with_ollama(prompt, model):
    result = subprocess.run(
        ["ollama", "run", model, prompt],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )
    if result.returncode != 0:
        raise RuntimeError(f"Ollama error: {result.stderr}")
    return result.stdout.strip()

In [11]:
#Universal function to get results of all metrics
def evaluate_metrics(text):
    return {
        "Flesch-Kincaid": textstat.flesch_kincaid_grade(text),
        "Flesch Ease": textstat.flesch_reading_ease(text),
        "ARI": textstat.automated_readability_index(text),
        "SMOG": textstat.smog_index(text),
        "Gunning Fog": textstat.gunning_fog(text),
        "Coleman-Liau": textstat.coleman_liau_index(text),
        "Spache": textstat.spache_readability(text),
        "Linsear Write": textstat.linsear_write_formula(text),
    }

In [13]:
# Defined readability levels. I decided to group school grades into school levels.
READABILITY_LEVELS = [
    "Early Elementary",
    "Upper Elementary",
    "Middle School",
    "High School",
    "College",
    "Advanced College",
    "Graduate",
    "Research"
]

FLESCH_KINCAID_GRADE_TARGET = {
    "Early Elementary": 2,
    "Upper Elementary": 5,
    "Middle School": 8,
    "High School": 11,
    "College": 13,
    "Advanced College": 15,
    "Graduate": 17,
    "Research": 18
}

In [26]:
#Functions for different prompting techniques
def zero_shot(topics_zh, readability_levels_zh, output_lengths_zh, model):
    results = []
    for topic in tqdm(topics_zh, desc="Topics"):
        for level_label in readability_levels_zh:
            for output_length in output_lengths_zh:
                prompt = (
                    f"Generate a short paragraph about '{topic}' targeting readability level "
                    f"{level_label}. Limit the output to {output_length} words."
                )

                print(f"\nPrompt: {prompt}")
                try:
                    output = generate_with_ollama(prompt, model)
                    metrics = evaluate_metrics(output)

                    print(f"Output:\n{output}")
                    print("Metrics:")
                    for k, v in metrics.items():
                        print(f"  {k}: {v:.2f}")

                    results.append({
                        "Technique": "Zero-Shot",
                        "Topic": topic,
                        "Target Level": level_label,
                        "Output Length": output_length,
                        "Model": model,
                        "Prompt": prompt,
                        "Rewrite Prompt": "",
                        "Text": output,
                        **metrics
                    })

                except Exception as e:
                    print(f"Error: {e}")
    return results

def few_shot(topics_fs, readability_levels_fs, output_lengths_fs, model, example_df):
    # Few-Shot Prompting (with examples from `excerpt_df`)
    results_few_shot = []
    for topic in tqdm(topics_fs, desc="Few-Shot"):
        for level_label in readability_levels_fs:
            # Retrieve 1–2 examples for the current level
            examples_df = example_df[example_df["Readability Level"] == level_label]
            few_shot_examples = examples_df["Excerpt"].sample(n=min(2, len(examples_df)), random_state=42).tolist()
            example_text = "\n\n".join([f"Example {i+1}:\n{ex}" for i, ex in enumerate(few_shot_examples)])

            for output_length in output_lengths_fs:
                prompt = (
                    f"{example_text}\n\n"
                    f"Using vocabulary and structure from examples, write a short paragraph about '{topic}' "
                    f"targeting the same readability level ({level_label}), "
                    f"limited to {output_length} words."
                )
                print(f"\nPrompt: {prompt}")
                try:
                    output = generate_with_ollama(prompt, model)
                    metrics = evaluate_metrics(output)

                    print(f"Output:\n{output}")
                    print("Metrics:")
                    for k, v in metrics.items():
                        print(f"  {k}: {v:.2f}")

                    results_few_shot.append({
                        "Technique": "Few-Shot",
                        "Topic": topic,
                        "Target Level": level_label,
                        "Output Length": output_length,
                        "Model": model,
                        "Prompt": prompt,
                        "Rewrite Prompt": "",
                        "Text": output,
                        **metrics
                    })
                except Exception as e:
                    print(f"Error: {e}")
    return results_few_shot

def two_step(topics_ts, readability_levels_ts, output_lengths_ts, model):
    # Two-step Prompting. First we generate text about topic, then rewrite it to readability level.
    results_two_step = []
    for topic in tqdm(topics_ts, desc="Two-step"):
        for level_label in readability_levels_ts:
            for output_length in output_lengths_ts:
                try:
                    summary_prompt = f"Write a summary about '{topic}'. Limit the output to {output_length} words."
                    summary = generate_with_ollama(summary_prompt, model)

                    print(f"\nPrompt: {summary_prompt}")

                    rewrite_prompt = (
                        f"Rewrite the following text to match the readability level of {level_label} "
                        f"and limit the output to {output_length} words:\n\n{summary}"
                    )

                    print(f"\nRewrite Prompt: {rewrite_prompt}")

                    output = generate_with_ollama(rewrite_prompt, model)
                    metrics = evaluate_metrics(output)

                    print(f"Output:\n{output}")
                    print("Metrics:")
                    for k, v in metrics.items():
                        print(f"  {k}: {v:.2f}")

                    results_two_step.append({
                        "Technique": "Two-Step",
                        "Topic": topic,
                        "Target Level": level_label,
                        "Output Length": output_length,
                        "Model": model,
                        "Prompt": summary_prompt,
                        "Rewrite Prompt": rewrite_prompt,
                        "Text": output,
                        **metrics
                    })
                except Exception as e:
                    print(f"Error: {e}")
    return results_two_step

def cot(topics_cot, readability_levels_cot, output_lengths_cot, model):
    # Chain-of-Thought Prompting. Text is generated step by step
    results_cot = []
    for topic in tqdm(topics_cot, desc="Chain-of-Thought"):
        for level_label in readability_levels_cot:
            for output_length in output_lengths_cot:
                prompt = (
                    f"Step 1: Think about how to explain '{topic}' to someone reading at the {level_label} level.\n"
                    f"Step 2: Select appropriate vocabulary and sentence structure.\n"
                    f"Step 3: Write a paragraph of {output_length} words that follows your plan."
                )

                print(f"\nPrompt: {prompt}")

                try:
                    output = generate_with_ollama(prompt, model)
                    metrics = evaluate_metrics(output)

                    print(f"Output:\n{output}")
                    print("Metrics:")
                    for k, v in metrics.items():
                        print(f"  {k}: {v:.2f}")

                    results_cot.append({
                        "Technique": "Chain-of-Thought",
                        "Topic": topic,
                        "Target Level": level_label,
                        "Output Length": output_length,
                        "Model": model,
                        "Prompt": prompt,
                        "Rewrite Prompt": "",
                        "Text": output,
                        **metrics
                    })
                except Exception as e:
                    print(f"Error: {e}")
    return results_cot

def cot_advanced(results_cot_adv, topics_cot, readability_levels_cot, output_lengths_cot, grade_target, model):
    # Chain-of-Thought Prompting. Text is generated step by step
    c = 0
    for topic in tqdm(topics_cot, desc="Chain-of-Thought"):
        topic_results = []

        for level_label in readability_levels_cot:
            for output_length in output_lengths_cot:
                level_numeric = grade_target[level_label]
                prompt = (
                    f"Step 1: Think about how to explain '{topic}' in {output_length} words.\n"
                    f"Step 2: Select appropriate vocabulary and sentence structure, so it will target "
                    f"Flesch-Kincaid Grade Level of approximately {level_numeric}. "
                    f"The Flesch-Kincaid Grade Level is calculated as:\n"
                    f"0.39 × (total words / total sentences) + 11.8 × (total syllables / total words) - 15.59.\n"
                    f"Step 3: Make sure the output has correct metric value.\n"
                    f"Step 4: Follow your plan and write the final paragraph."
                )

                try:
                    output = generate_with_ollama(prompt, model)
                    metrics = evaluate_metrics(output)

                    result = {
                        "Technique": "Chain-of-Thought-Advanced",
                        "Topic": topic,
                        "Target Level": level_label,
                        "Output Length": output_length,
                        "Model": model,
                        "Prompt": prompt,
                        "Rewrite Prompt": "",
                        "Text": output,
                        **metrics
                    }

                    results_cot_adv.append(result)
                    topic_results.append(result)
                    c += 1
                    print(c)
                except Exception as e:
                    print(f"Error: {e}")

        # Save topic_results to a CSV file after processing all combinations for the topic
        safe_topic_name = "".join(c if c.isalnum() or c in (' ', '-', '_') else '_' for c in topic).replace(' ', '_')
        output_path = f"{safe_topic_name}.csv"
        pd.DataFrame(topic_results).to_csv(output_path, index=False)

    return results_cot_adv

In [1]:
#Experiment implementation
results_zh_mistral = zero_shot(TOPICS, READABILITY_LEVELS, OUTPUT_LENGTHS, model_mistral)
results_zh_mistral_df = pd.DataFrame(results_zh_mistral)
results_zh_mistral_df.to_csv("mistral_readability_results_zero-shot.csv", index=False)

NameError: name 'zero_shot' is not defined

In [56]:
results_fs_mistral = few_shot(TOPICS, READABILITY_LEVELS, OUTPUT_LENGTHS, model_mistral, excerpt_df)
results_fs_mistral_df = pd.DataFrame(results_fs_mistral)
results_fs_mistral_df.to_csv("mistral_readability_results_few-shot.csv", index=False)

Few-Shot:   0%|                                           | 0/5 [00:00<?, ?it/s]


Prompt: Example 1:
Cat and Dog walk. They walk in their village. Then they see an egg. The egg is in the grass. The egg is alone in the grass. The egg is all alone. Cat and Dog walk to a bird. They ask the bird, "Is this your egg?" But the bird says, "No, that is not my egg. Ask the owl. Maybe it is his egg." Cat and Dog walk to the owl. They ask the owl, "Is this your egg?" But the owl says, "No, that is not my egg. Ask the goose. Maybe it is her egg." Cat and Dog walk to the goose. They ask the goose, "Is this your egg?" But the goose says, "No, that is not my egg. Ask the two ducks. Maybe it is their egg." Cat and Dog walk to the two ducks. They ask the two ducks, "Is this your egg?" But the two ducks say, "No, that is not our egg." Then the egg breaks.

Example 2:
The Sun’s light shines on Earth. The light makes our air, land, and water warm. But some things get
warmer than others.
Pretend it is a sunny day. If you wore a black shirt, you would feel hot. But if you wore a white sh

python(75810) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Output:
A person grows older, just like how a seed becomes a big tree. As time passes, they may get gray hair and wrinkles. This is called getting old or human aging. Some people live longer than others, but all people eventually get very old. Like the egg asking animals about its origin, we can't stop the passage of time, but we can make choices to stay healthy!
Metrics:
  Flesch-Kincaid: 6.02
  Flesch Ease: 75.80
  ARI: 5.73
  SMOG: 7.55
  Gunning Fog: 7.15
  Coleman-Liau: 6.23
  Spache: 4.53
  Linsear Write: 6.30

Prompt: Example 1:
Cat and Dog walk. They walk in their village. Then they see an egg. The egg is in the grass. The egg is alone in the grass. The egg is all alone. Cat and Dog walk to a bird. They ask the bird, "Is this your egg?" But the bird says, "No, that is not my egg. Ask the owl. Maybe it is his egg." Cat and Dog walk to the owl. They ask the owl, "Is this your egg?" But the owl says, "No, that is not my egg. Ask the goose. Maybe it is her egg." Cat and Dog walk to

python(75812) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


...

Output:
Nutrition, a vital aspect of human survival, dates back to ancient civilizations. For millennia, it remained a crucial element in sustaining life, yet its scientific understanding was minimal. By the end of the 18th Century, researchers began studying nutrition systematically, leading to significant advancements in the 19th century. Nutrition science is fundamental in understanding our dietary needs and maintaining health. Food is transformed mechanically into nutritional energy within our bodies, which powers various biological functions. This process occurs through digestion, where complex molecules are broken down into simpler ones, making them accessible for absorption. The primary method of obtaining nutrients is either ingesting foods rich in essential elements (the triboelectric effect, or consumption) or inducing nutrient production within our bodies through specific dietary choices (electrostatic induction). Proper nutrition is crucial to ensure a balanced and healthy 

python(76277) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Output:
Nutrition, a fundamental aspect of life, dates back to ancient civilizations, where it was primarily understood as a means to sustain energy levels. However, it wasn't until modern times that nutrition evolved into a comprehensive science. The study of nutrition began in earnest during the 19th century, contributing significantly to understanding the role of food in maintaining health and preventing diseases. Nutrition operates by transforming ingested food into nutrients, using biological processes within our bodies. This process involves digestion, absorption, transportation, and metabolism of various macronutrients (proteins, carbohydrates, fats) and micronutrients (vitamins, minerals). These nutrients are essential for growth, repair, energy production, and immune system functioning. Two primary methods of obtaining these nutrients are ingestion (consumption) or synthesis (production within the body). Just as electrostatic generators develop charges through mechanical work,

python(76278) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Few-Shot: 100%|██████████████████████████████████| 5/5 [35:36<00:00, 427.25s/it]

Output:
The understanding of nutrition, as a vital part of human health, dates back to ancient civilizations, serving primarily as means to sustain life. However, it was not until the 19th century that the scientific study of nutrition truly emerged. Early researchers focused on identifying essential nutrients and their roles in maintaining bodily functions. The development of modern nutrition did not begin in earnest until the 20th century, when it became a fundamental discipline in medical and health sciences. Nutrition operates by providing our bodies with the necessary energy, macronutrients, micronutrients, and water to function optimally. This process involves transforming the mechanical act of consuming food into chemical energy through digestion and metabolism.

The body generates this energy through two primary methods: catabolic reactions (breaking down complex molecules) or anabolic reactions (building up simple molecules). Just like electrostatic generators, our bodies are 

In [57]:
results_ts_mistral = two_step(TOPICS, READABILITY_LEVELS, OUTPUT_LENGTHS, model_mistral)
results_ts_mistral_df = pd.DataFrame(results_ts_mistral)
results_ts_mistral_df.to_csv("mistral_readability_results_two-step.csv", index=False)

Two-step:   0%|                                           | 0/5 [00:00<?, ?it/s]python(76279) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.



Prompt: Write a summary about 'human aging'. Limit the output to 50 words.

Rewrite Prompt: Rewrite the following text to match the readability level of Early Elementary and limit the output to 50 words:

Human aging, a complex process influenced by genetics and environmental factors, results in decline across various physiological systems over time. It involves changes at the cellular level, such as telomere shortening and alterations in gene expression, leading to tissue degeneration and functional impairments. Understanding this process can pave the way for effective interventions and strategies to prolong healthspan.


python(76281) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Output:
People get older with time, this happens because of things we are born with (genes) and where we live. It makes our body work less well in many ways. This includes changes in our cells like shorter telomeres and different genes working. This causes parts of the body to wear out and stop working as well. Learning about this helps us find ways to keep feeling good for longer.
Metrics:
  Flesch-Kincaid: 5.53
  Flesch Ease: 80.03
  ARI: 6.49
  SMOG: 6.74
  Gunning Fog: 6.10
  Coleman-Liau: 7.27
  Spache: 4.28
  Linsear Write: 6.30


python(76282) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


...

Output:
Life's foundation lies in proper nutrition, which involves consuming necessary food and drinks for sustenance and growth. Balanced diets require proteins, carbs, fats, vitamins, minerals, and water. Each nutrient performs a unique role: proteins repair and build body tissues, carbs supply energy, fats store it, vitamins boost metabolism, and minerals strengthen bones and perform other functions. Eating a diverse range of foods ensures the body gets essential nutrients. Regular exercise, portion control, and maintaining a healthy weight further promote good nutrition. Inadequate or excessive nutrition can lead to malnutrition or diseases like obesity, diabetes, and heart disease. Cultivating awareness, gaining knowledge, and making wise choices are crucial for developing sound eating habits.
Metrics:
  Flesch-Kincaid: 13.09
  Flesch Ease: 29.70
  ARI: 15.59
  SMOG: 14.76
  Gunning Fog: 16.76
  Coleman-Liau: 16.79
  Spache: 7.24
  Linsear Write: 11.14


python(77651) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.



Prompt: Write a summary about 'nutrition'. Limit the output to 200 words.

Rewrite Prompt: Rewrite the following text to match the readability level of Research and limit the output to 200 words:

Nutrition refers to the process of consuming and utilizing food substances to sustain life, promote growth, and maintain health. A balanced diet should include a variety of foods from all food groups: fruits, vegetables, grains, proteins (meat, poultry, fish, eggs, legumes), and dairy or dairy alternatives. Nutrients, such as carbohydrates, proteins, fats, vitamins, and minerals, are essential for our bodies to function properly. Macronutrients provide energy, while micronutrients (vitamins and minerals) are required in small amounts but are vital for numerous bodily functions.

Maintaining a healthy diet is crucial for overall well-being. It can help prevent chronic diseases like heart disease, diabetes, and certain types of cancer. Proper nutrition also supports the immune system, boosts e

python(77666) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Two-step: 100%|███████████████████████████████| 5/5 [3:20:03<00:00, 2400.75s/it]

Output:
Food and Nutrition: Essentials for Life

Food and nutrition play vital roles in sustaining life, fostering growth, and preserving health. To ensure a balanced diet, consume foods from all groups: fruits, vegetables, grains, proteins (meat, poultry, fish, eggs, legumes), and dairy or dairy alternatives. The body needs essential nutrients like carbohydrates, proteins, fats, vitamins, and minerals to function correctly. While energy-providing macronutrients are necessary, micronutrients (vitamins and minerals) are crucial for various bodily functions despite being required in small amounts.

Maintaining a healthy diet is essential for overall well-being. It helps prevent chronic diseases such as heart disease, diabetes, and specific types of cancer. Proper nutrition strengthens the immune system, raises energy levels, and supports mental health. Conversely, poor nutrition may result in malnutrition, obesity, and several health complications.

Individual nutritional requirements ca

In [58]:
results_cot_mistral = cot(TOPICS, READABILITY_LEVELS, OUTPUT_LENGTHS, model_mistral)
results_cot_mistral_df = pd.DataFrame(results_cot_mistral)
results_cot_mistral_df.to_csv("mistral_readability_results_cot.csv", index=False)

Chain-of-Thought:   0%|                                   | 0/5 [00:00<?, ?it/s]


Prompt: Step 1: Think about how to explain 'human aging' to someone reading at the Early Elementary level.
Step 2: Select appropriate vocabulary and sentence structure.
Step 3: Write a paragraph of 50 words that follows your plan.


python(77667) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Output:
Aging is like a special clock in our bodies. As this clock ticks, we grow older. Every year, our bodies create new cells to help us grow and play. However, over time, these cells sometimes make mistakes or get damaged. When this happens, our bodies can't fix them as well as they used to, which leads to wrinkles, gray hair, and other signs of aging. But remember, getting older doesn't mean we become less important or valuable! We are all unique and special at any age.
Metrics:
  Flesch-Kincaid: 5.80
  Flesch Ease: 75.33
  ARI: 5.85
  SMOG: 7.96
  Gunning Fog: 4.91
  Coleman-Liau: 6.61
  Spache: 3.77
  Linsear Write: 5.86

Prompt: Step 1: Think about how to explain 'human aging' to someone reading at the Early Elementary level.
Step 2: Select appropriate vocabulary and sentence structure.
Step 3: Write a paragraph of 100 words that follows your plan.


python(77669) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


...

Output:
At the research level, nutrition refers to the study of nutrients in food, their absorption, transport, metabolism, and excretion in living organisms. It encompasses understanding the chemical composition of foods, their effects on body function, and the role they play in maintaining health and preventing disease. Nutrition science investigates macronutrients (carbohydrates, proteins, fats, and water), micronutrients (vitamins and minerals), dietary fiber, and other bioactive compounds that influence our physiology. It also delves into the principles of energy balance, nutrient requirements, and their impact on body weight management and overall health. Moreover, it examines the interplay between genetic factors, lifestyle choices, and environmental influences in determining an individual's nutritional needs and responses to different dietary patterns. In essence, nutrition research seeks to provide evidence-based recommendations for optimizing human health and performance thro

python(78915) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Output:
Nutrition, in the context of research, refers to the study of nutrients in food, how they affect our bodies, and the processes by which we absorb and utilize them. It encompasses the science of food, its biological components, and its impact on growth, development, and overall health. This field delves into macronutrients (carbohydrates, proteins, fats, water, and minerals), micronutrients (vitamins and trace elements), and their roles in various metabolic pathways.

At the research level, nutrition is not just about consuming a balanced diet but understanding the molecular mechanisms behind nutrient absorption, distribution, storage, and excretion. It involves the exploration of dietary patterns, their effects on gene expression, and the potential for personalized nutrition based on genetic predispositions.

Moreover, nutrition research investigates the intricate relationships between diet and disease, including obesity, diabetes, cardiovascular diseases, and certain types of 

python(78919) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Chain-of-Thought: 100%|███████████████████████| 5/5 [4:00:10<00:00, 2882.15s/it]

Output:
Title: Nutrition in the Context of Advanced Research

   In the realm of scientific study, nutrition stands as a multidisciplinary field that delves into the intricate interplay between food, nutrients, and the body's processes. It encompasses the ingestion, absorption, transport, metabolism, and excretion of essential substances required for growth, maintenance, and repair of an organism.

   At the research level, the focus lies on understanding the macronutrients (carbohydrates, proteins, fats, water, and dietary fiber) and micronutrients (vitamins and minerals), their respective roles, and interactions within the body. This includes investigating how these nutrients support biological functions such as energy production, immune response, structural integrity, hormone production, and numerous other physiological processes.

   More sophisticated research methods are employed to explore the impact of dietary patterns on health outcomes, disease development, and treatment effi

In [59]:
results_zh_llama = zero_shot(TOPICS, READABILITY_LEVELS, OUTPUT_LENGTHS, model_llama)
results_zh_llama_df = pd.DataFrame(results_zh_llama)
results_zh_llama_df.to_csv("llama_readability_results_zero-shot.csv", index=False)

Topics:   0%|                                             | 0/5 [00:00<?, ?it/s]


Prompt: Generate a short paragraph about 'human aging' targeting readability level Early Elementary. Limit the output to 50 words.


python(78920) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Output:
Our bodies get older just like toys do! When we're babies, our skin is soft and stretchy. As we grow up, our skin gets wrinkly and slower. Our hair might turn gray or fall out. We also get weaker and less flexible. This is called growing old.
Metrics:
  Flesch-Kincaid: 2.53
  Flesch Ease: 90.88
  ARI: 2.13
  SMOG: 5.46
  Gunning Fog: 3.98
  Coleman-Liau: 3.50
  Spache: 2.86
  Linsear Write: 3.08

Prompt: Generate a short paragraph about 'human aging' targeting readability level Early Elementary. Limit the output to 100 words.


python(78923) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


...

Output:
Nutrition is the process by which organisms acquire and utilize nutrients from their environment to sustain life. It involves the ingestion, digestion, absorption, and metabolism of macronutrients (carbohydrates, fats, and proteins) and micronutrients (vitamins and minerals). Adequate nutrition is essential for growth, development, and maintenance of bodily functions. Poor nutrition can lead to various health issues such as obesity, diabetes, and cardiovascular disease. Nutritional needs vary across individuals based on factors like age, sex, weight, and physical activity level. A balanced diet with proper nutrient intake is crucial for overall well-being and optimal health outcomes.
Metrics:
  Flesch-Kincaid: 14.93
  Flesch Ease: 16.22
  ARI: 16.13
  SMOG: 16.73
  Gunning Fog: 20.65
  Coleman-Liau: 17.44
  Spache: 7.42
  Linsear Write: 13.58

Prompt: Generate a short paragraph about 'nutrition' targeting readability level Research. Limit the output to 150 words.


python(79760) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Output:
Here is a 150-word paragraph on "nutrition" targeting a readability level of Research:

Nutrition refers to the process by which living organisms utilize energy and nutrients to sustain life. The study of nutrition involves understanding the complex interactions between food, environment, and physiology, as well as the biochemical processes that govern energy metabolism. Macronutrients such as carbohydrates, proteins, and fats provide essential energy sources, while micronutrients including vitamins and minerals regulate various physiological functions. Adequate nutrition is crucial for maintaining optimal health, preventing chronic diseases, and supporting growth and development. Malnutrition, on the other hand, can lead to impaired immune function, decreased productivity, and increased morbidity and mortality. Advances in nutritional science have led to a better understanding of the relationship between diet and disease, with emerging evidence highlighting the importance of p

python(79761) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Topics: 100%|█████████████████████████████████| 5/5 [2:33:54<00:00, 1846.88s/it]

Output:
Nutrition is the process by which organisms acquire and utilize energy and nutrients for growth, maintenance, and repair of bodily functions. It encompasses the study of how different components in food interact with the body, influencing overall health and disease prevention. A well-balanced diet provides essential macronutrients such as carbohydrates, proteins, and fats, alongside micronutrients like vitamins and minerals that are vital for cellular processes.

The human body requires a broad range of nutrients to maintain optimal functioning, including energy-producing compounds, growth factors, and protective agents against oxidative stress. Nutritional deficiencies can lead to various health issues, from mild disorders like fatigue and weakness to severe conditions such as anemia and even mental health impairments. Conversely, consuming adequate amounts of the right nutrients supports immune function, cognitive performance, and overall well-being.

Effective nutrition is a

In [60]:
results_fs_llama = few_shot(TOPICS, READABILITY_LEVELS, OUTPUT_LENGTHS, model_llama, excerpt_df)
results_fs_llama_df = pd.DataFrame(results_fs_llama)
results_fs_llama_df.to_csv("llama_readability_results_few-shot.csv", index=False)

Few-Shot:   0%|                                           | 0/5 [00:00<?, ?it/s]


Prompt: Example 1:
Cat and Dog walk. They walk in their village. Then they see an egg. The egg is in the grass. The egg is alone in the grass. The egg is all alone. Cat and Dog walk to a bird. They ask the bird, "Is this your egg?" But the bird says, "No, that is not my egg. Ask the owl. Maybe it is his egg." Cat and Dog walk to the owl. They ask the owl, "Is this your egg?" But the owl says, "No, that is not my egg. Ask the goose. Maybe it is her egg." Cat and Dog walk to the goose. They ask the goose, "Is this your egg?" But the goose says, "No, that is not my egg. Ask the two ducks. Maybe it is their egg." Cat and Dog walk to the two ducks. They ask the two ducks, "Is this your egg?" But the two ducks say, "No, that is not our egg." Then the egg breaks.

Example 2:
The Sun’s light shines on Earth. The light makes our air, land, and water warm. But some things get
warmer than others.
Pretend it is a sunny day. If you wore a black shirt, you would feel hot. But if you wore a white sh

python(79762) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Output:
Here is a paragraph about human aging:

Our bodies change as we grow older. Our skin gets wrinkles and our hair turns gray or white. We might walk slower or need glasses. But some things don't change. We still have fun with friends, play games, or read books. As we age, our hearts beat slower.
Metrics:
  Flesch-Kincaid: 3.00
  Flesch Ease: 89.86
  ARI: 3.62
  SMOG: 5.46
  Gunning Fog: 4.39
  Coleman-Liau: 5.01
  Spache: 3.23
  Linsear Write: 3.75

Prompt: Example 1:
Cat and Dog walk. They walk in their village. Then they see an egg. The egg is in the grass. The egg is alone in the grass. The egg is all alone. Cat and Dog walk to a bird. They ask the bird, "Is this your egg?" But the bird says, "No, that is not my egg. Ask the owl. Maybe it is his egg." Cat and Dog walk to the owl. They ask the owl, "Is this your egg?" But the owl says, "No, that is not my egg. Ask the goose. Maybe it is her egg." Cat and Dog walk to the goose. They ask the goose, "Is this your egg?" But the goo

python(79777) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


...

Output:
The study of nutrition dates back to ancient civilizations, yet for centuries it remained an enigmatic phenomenon without a comprehensive theory to explain its principles. Not until the 18th century did researchers develop practical means of understanding nutritional requirements by transforming knowledge of food science into actionable guidelines. Nutritionists operate by using empirical data to transform quantitative analysis into qualitative insights. The study of nutrition involves analyzing macronutrients and micronutrients, with work divided between two branches: one dealing with energy sources (calories) and the other with essential nutrients (vitamins). Charges are generated through scientific induction and measurement.
Metrics:
  Flesch-Kincaid: 17.52
  Flesch Ease: 3.99
  ARI: 19.39
  SMOG: 18.46
  Gunning Fog: 22.61
  Coleman-Liau: 20.28
  Spache: 8.10
  Linsear Write: 16.90

Prompt: Example 1:
An electrostatic generator, or electrostatic machine, is an electromechan

python(80332) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Output:
Example:

Nutritional systems have been understood for centuries, yet only recently has their behavior been thoroughly explained. The study of nutrition as we know it today began in earnest during the late 19th century with the discovery that food is composed of various chemical compounds. Early researchers had developed methods to analyze nutrient content using simple laboratory equipment, but a comprehensive theory of nutrition did not emerge until the early 20th century. Nutrition scientists operate by transforming basic biological processes into empirical data through rigorous experimentation and observation. The knowledge of nutritional science develops by analyzing the effects of individual nutrients on living organisms, often using statistical models to predict outcomes. Researchers have developed two primary methods for studying nutrition: either direct measurement of nutrient intake or analysis of physiological responses to varying levels of nutrient consumption.

This

python(80334) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Few-Shot: 100%|██████████████████████████████████| 5/5 [43:40<00:00, 524.12s/it]

Output:
Here is a paragraph on "nutrition" written in a style similar to Examples 1 and 2:

The study of nutrition has its roots in ancient civilizations, where knowledge of food's impact on human health was often anecdotal and without theoretical basis. However, by the late 19th century, researchers had developed practical means of measuring the nutritional value of foods, laying the foundation for a scientific understanding of the subject. Nutritionists today operate by converting biochemical processes into quantifiable energy values, using various methods to determine the caloric content of different food groups. This is typically achieved through manual analysis of macronutrients (carbohydrates, proteins, and fats) or via instrumental techniques like chromatography. Two primary approaches exist: either analyzing chemical reactions that release energy or measuring heat changes associated with digestion. The charge on nutrition as a field has been generated by advances in biochemistr

In [61]:
results_ts_llama = two_step(TOPICS, READABILITY_LEVELS, OUTPUT_LENGTHS, model_llama)
results_ts_llama_df = pd.DataFrame(results_ts_llama)
results_ts_llama_df.to_csv("llama_readability_results_two-step.csv", index=False)

Two-step:   0%|                                           | 0/5 [00:00<?, ?it/s]python(80335) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.



Prompt: Write a summary about 'human aging'. Limit the output to 50 words.

Rewrite Prompt: Rewrite the following text to match the readability level of Early Elementary and limit the output to 50 words:

Human aging is a complex, multifaceted process characterized by gradual decline in physical and biological functions. It involves accumulation of cellular damage, epigenetic changes, and dysregulation of metabolic pathways, leading to reduced resilience, increased disease susceptibility, and eventual mortality. Aging affects various bodily systems, including energy production, growth, and repair.


python(80337) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Output:
Here is the rewritten text:

As we get older, our bodies start to slow down. It's a natural process that happens gradually. Our cells get hurt, and it gets harder for them to work together properly. This can lead to getting sick more easily and eventually being very old or tired.
Metrics:
  Flesch-Kincaid: 7.66
  Flesch Ease: 62.85
  ARI: 6.09
  SMOG: 10.69
  Gunning Fog: 9.81
  Coleman-Liau: 7.01
  Spache: 4.32
  Linsear Write: 7.12


python(80338) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


...

Output:
Here is the rewritten text at a readability level similar to research articles:

Nutritional processes involve the acquisition and utilization of energy and nutrients by living organisms through the consumption of food. This process plays a pivotal role in maintaining homeostasis, facilitating growth and development, and supporting overall physiological function.

A balanced diet comprising essential macronutrients (carbohydrates, proteins, fats) and micronutrients (vitamins, minerals) is indispensable for optimising bodily functions such as metabolic rate, immune response, and cellular repair. Adequate nutrition is also crucial for preventing chronic conditions like obesity, diabetes, cardiovascular disease, and certain malignancies.

Factors influencing nutritional status include genetic predisposition, lifestyle factors (exercise, sleep patterns), and environmental determinants (access to clean water and nutrient-dense food). Optimal nutrition is essential for maintaining pe

python(80988) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.



Prompt: Write a summary about 'nutrition'. Limit the output to 200 words.

Rewrite Prompt: Rewrite the following text to match the readability level of Research and limit the output to 200 words:

Nutrition is the process by which organisms obtain and utilize energy and essential nutrients from their environment to sustain life. It involves the consumption of food, absorption, and utilization of nutrients such as carbohydrates, proteins, fats, vitamins, and minerals.

Proper nutrition is crucial for maintaining optimal health, growth, and development. A well-balanced diet provides the necessary fuel for bodily functions, including energy production, growth and repair of tissues, and maintenance of bodily systems.

Nutritional deficiencies can lead to a range of health problems, from mild to severe. For example, a lack of vitamin D can cause rickets in children, while inadequate iron intake can result in anemia.

Understanding nutrition involves knowing the recommended daily intake of 

python(80990) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Two-step: 100%|████████████████████████████████| 5/5 [1:21:41<00:00, 980.21s/it]

Output:
Here's the rewritten text, condensed to 200 words and tailored to match the readability level of research papers:

**Nutritional Requirements for Optimal Health**

Nutrition is the process by which organisms acquire and utilize energy and essential nutrients from their environment to sustain life. This involves consuming food, absorbing, and utilizing macronutrients (carbohydrates, proteins, fats) and micronutrients (vitamins, minerals). Proper nutrition is vital for maintaining optimal health, growth, and development.

A well-balanced diet provides the necessary fuel for bodily functions, including energy production, tissue growth, and maintenance of physiological systems. Nutritional deficiencies can lead to a range of health problems, from mild to severe. For example, vitamin D deficiency can cause rickets in children, while inadequate iron intake can result in anemia.

Understanding nutrition involves knowing the recommended daily intake of essential nutrients, which varies

In [62]:
results_cot_llama = cot(TOPICS, READABILITY_LEVELS, OUTPUT_LENGTHS, model_llama)
results_cot_llama_df = pd.DataFrame(results_cot_llama)
results_cot_llama_df.to_csv("llama_readability_results_cot.csv", index=False)

Chain-of-Thought:   0%|                                   | 0/5 [00:00<?, ?it/s]


Prompt: Step 1: Think about how to explain 'human aging' to someone reading at the Early Elementary level.
Step 2: Select appropriate vocabulary and sentence structure.
Step 3: Write a paragraph of 50 words that follows your plan.


python(80991) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Output:
Here's my attempt:

"As we grow up, our bodies change in many ways. Our skin gets wrinkly, our hair turns gray or falls out, and we lose some strength. This is called growing old. It happens because our cells get older and start to work less well. But even when we're old, our hearts still beat and our minds are sharp!"

I tried to use vocabulary that an Early Elementary reader would understand, such as "wrinkly", "gray or falls out", and "cells". I also kept the sentence structure simple, with short sentences and basic phrases like "As we grow up..." and "This is called...". Finally, I aimed for a paragraph length of around 50 words. Let me know if you have any feedback!
Metrics:
  Flesch-Kincaid: 4.84
  Flesch Ease: 82.24
  ARI: 5.97
  SMOG: 7.55
  Gunning Fog: 6.22
  Coleman-Liau: 6.13
  Spache: 3.41
  Linsear Write: 6.71

Prompt: Step 1: Think about how to explain 'human aging' to someone reading at the Early Elementary level.
Step 2: Select appropriate vocabulary and senten

python(80992) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


...

Output:
Here's an explanation of nutrition suitable for a research-level audience:

Nutrition encompasses the comprehensive study of how organisms utilize and interact with their dietary components to sustain growth, maintenance, and reproduction. It involves the analysis of nutrient uptake, metabolism, and utilization across various physiological systems, including energy production, tissue repair, and immune function. The discipline integrates principles from biochemistry, physiology, and molecular biology to understand the intricate relationships between diet, health, and disease. By investigating the complexities of nutritional biochemistry and its impact on human and animal health, researchers aim to develop evidence-based recommendations for optimal nutrition.

(Note: I've aimed for a level of vocabulary and sentence structure that is typical in academic writing at the research level, with a focus on technical terms and concepts.)
Metrics:
  Flesch-Kincaid: 20.55
  Flesch Ease: -

python(81454) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Output:
Here's my approach:

**Step 1: Define "nutrition" for a research-level audience**

To explain "nutrition" to someone reading at the Research level, I'll need to provide a detailed and technical description. This will involve discussing the physiological processes involved in nutrient uptake and utilization by the body.

**Step 2: Select vocabulary and sentence structure**

For a research-level audience, I'll use precise and specialized language, including technical terms and concepts from nutrition science. Sentence structures will be formal and objective, with complex sentences and nuanced explanations.

**Step 3: Write the paragraph**

Nutrition refers to the biochemical processes involved in the ingestion, absorption, assimilation, and utilization of nutrients by living organisms. This encompasses a wide range of physiological mechanisms, including nutrient transport across cell membranes, enzymatic digestion and processing, and energy metabolism. The intricate relationships

python(81455) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Chain-of-Thought: 100%|██████████████████████████| 5/5 [58:56<00:00, 707.38s/it]

Output:
Here's an explanation of nutrition suitable for a reader at the research level:

Nutrition is the interdisciplinary field concerned with understanding how individuals obtain, process, and utilize nutrients to sustain life and maintain optimal physiological function. It encompasses the study of dietary components, including macronutrients (carbohydrates, proteins, fats) and micronutrients (vitamins, minerals), as well as their interactions within the human body. Nutrition research explores the effects of nutrient intake on growth, development, and health outcomes across various populations, from fetal development to older adulthood. The field also examines how environmental factors, such as socioeconomic status, access to food resources, and cultural practices, influence dietary patterns and nutritional status. Nutritional genomics is an emerging area within the field that investigates the interplay between genetic variation, nutrient metabolism, and disease susceptibility. By e

In [16]:
results_cot_adv_mistral = cot_advanced(TOPICS, READABILITY_LEVELS, OUTPUT_LENGTHS, FLESCH_KINCAID_GRADE_TARGET, model_mistral)
results_cot_adv_mistral_df = pd.DataFrame(results_cot_adv_mistral)
results_cot_adv_mistral_df.to_csv("mistral_readability_results_cot_advanced.csv", index=False)

Chain-of-Thought:   0%|                                     | 0/5 [00:00<?, ?it/s]


Prompt: Step 1: Think about how to explain 'human aging' in 50 words.
Step 2: Select appropriate vocabulary and sentence structure, so it will target Flesch-Kincaid Grade Level of approximately 2. The Flesch-Kincaid Grade Level is calculated as:
0.39 × (total words / total sentences) + 11.8 × (total syllables / total words) - 15.59.
Step 3: Make sure the output has correct metric value.
Step 4: Follow your plan and write the final paragraph.
Output:
Aging in humans happens slowly over time, much like a ripe fruit or an old tree. We grow taller when young, then stop, and gradually become smaller as we get older. Our skin changes color and may have wrinkles. Some parts of our body stop working well, like memory and strength. This is normal, but taking care of ourselves can help us stay healthy longer.
Metrics:
  Flesch-Kincaid: 6.18
  Flesch Ease: 73.90
  ARI: 6.23
  SMOG: 7.55
  Gunning Fog: 6.43
  Coleman-Liau: 7.09
  Spache: 4.39
  Linsear Write: 6.10

Prompt: Step 1: Think about how

Output:
Aging in humans is a natural process that happens over time, causing changes in our bodies. As we grow older, our cells may not repair or regenerate as well as they used to, leading to wear and tear. This includes changes in skin texture, hair color, and vision issues like farsightedness. Our metabolism slows down too, which means we burn calories less efficiently. Additionally, the immune system weakens, making us more prone to diseases. Regular exercise, a balanced diet, good sleep, and regular check-ups can help slow down some of these changes. Remember, everyone ages differently, so what happens to one person may not happen to another.

Flesch-Kincaid Grade Level: 5.3 (Total Words = 104, Total Sentences = 7, Syllables = 236)
Metrics:
  Flesch-Kincaid: 9.10
  Flesch Ease: 53.36
  ARI: 8.96
  SMOG: 11.21
  Gunning Fog: 10.33
  Coleman-Liau: 10.47
  Spache: 5.52
  Linsear Write: 8.14

Prompt: Step 1: Think about how to explain 'human aging' in 200 words.
Step 2: Select appropr

Output:
Aging in humans is a natural process where our bodies gradually change over time, leading to a decline in physical and mental abilities. This happens due to a combination of factors such as cellular damage from daily living, genetic predispositions, and environmental influences. As we grow older, our cells can't repair themselves as effectively, tissues break down, and organs lose their efficiency, ultimately leading to health issues associated with aging like wrinkles, grey hair, and chronic diseases.

Using the Flesch-Kincaid readability formula:
1. Total words = 38 (Aging in humans, ..., health issues)
2. Total sentences = 4
3. Syllables count: 'Aging' - 5, 'humans' - 2, 'natural process' - 3, 'gradually change' - 4, 'leading' - 2, 'decline' - 2, 'physical and mental abilities' - 5, 'This happens' - 3, 'combination' - 3, 'factors' - 1, 'such as' - 2, 'cellular damage' - 4, 'daily living' - 3, 'genetic predispositions' - 4, 'environmental influences' - 4, 'As we grow older', 

Output:
Aging in humans is a natural process that occurs over time, leading to changes in our bodies and eventually affecting our health. As we age, cells in our bodies divide and reproduce less efficiently, leading to a decrease in the number of healthy cells. This can result in weakened tissues and organs, making us more susceptible to diseases. Additionally, as we grow older, our metabolism slows down, causing us to gain weight more easily and have less energy. Other factors such as genetics, lifestyle, and environment also play a role in how fast or slow we age. However, understanding the process of aging can help us make informed decisions about maintaining our health and well-being as we grow older.

The Flesch-Kincaid Grade Level is approximately 13. The total words are 96, the total sentences are 5, the total syllables are 248. Therefore, the Flesch-Kincaid Grade Level is:
0.39 × (96 / 5) + 11.8 × (248 / 96) - 15.59 = 8.7

This means that the text can be easily understood by an

...

Output:
Nutrition refers to the process of consuming and utilizing food for growth, repair, and energy in our bodies. It involves eating a balanced diet that provides essential nutrients like proteins, carbohydrates, fats, vitamins, and minerals. These nutrients help build strong bones, boost immunity, maintain good health, and prevent diseases. The right nutrition can enhance overall well-being and quality of life.

According to Flesch-Kincaid Grade Level calculation:
The number of words: 32 (total words)
The number of sentences: 1 (total sentences)
Total syllables: 59 (total syllables)
Flesch-Kincaid Grade Level = 0.39 * (32/1) + 11.8 * (59/32) - 15.59 ≈ 12.7 (Rounded to the nearest whole number, this text is approximately at a 13th-grade level.)

The metric value of nutrients in food is typically measured in milligrams (mg), grams (g), or international units (IU) for vitamins. For example, calcium might be listed as 200 mg per serving, and Vitamin A as 150 IU per serving.
Metrics:
 

Output:
Nutrition is all about the food we eat and how it impacts our body. It's essential for our growth, repair, and overall health. We consume nutrients like proteins, carbohydrates, fats, vitamins, and minerals from food to give us energy and keep us strong. Balanced nutrition means getting the right amount of these nutrients each day. Eating a variety of foods helps ensure we get all the nutrients our bodies need. A healthy diet includes plenty of fruits, vegetables, lean proteins, and whole grains. Remember, portion sizes are important too! A balanced diet also means limiting foods that are high in sugar, salt, and unhealthy fats. It's crucial to talk to a healthcare provider if you have any questions about what you should eat. So, make wise food choices for a healthy life!

Flesch-Kincaid Grade Level: 5.2 (Estimated based on the formula provided)
Metrics:
  Flesch-Kincaid: 7.56
  Flesch Ease: 61.97
  ARI: 8.20
  SMOG: 10.86
  Gunning Fog: 10.37
  Coleman-Liau: 9.68
  Spache: 4.7

Output:
Nutrition, in simple terms, is about nourishing our bodies with the right food and drinks to sustain life, maintain good health, and boost energy levels. It involves consuming a balanced diet that provides essential nutrients such as carbohydrates, proteins, fats, vitamins, minerals, and water.

Foods rich in these nutrients help our bodies perform various functions effectively. For instance, carbohydrates are a primary source of energy, while proteins are building blocks for growth and repair of body tissues. Vitamins and minerals, on the other hand, play crucial roles in numerous bodily processes such as immune function, blood clotting, and bone health.

The key to good nutrition lies in moderation, variety, and balance. Eating a variety of foods helps ensure that you get all the necessary nutrients. Portion control is also essential to prevent overconsumption of certain nutrients that could potentially lead to health issues. Lastly, maintaining a balanced diet means not over

Chain-of-Thought: 100%|████████████████████████████| 5/5 [45:29<00:00, 545.86s/it]

Output:
Nutrition refers to the process of providing or obtaining the food necessary for the growth, maintenance, and repair of an organism. It involves consuming a variety of nutrients in appropriate amounts to maintain good health. These essential nutrients include proteins, carbohydrates, fats, vitamins, minerals, and water. They provide energy, help build and repair body tissues, support bodily functions, and defend against diseases.

Balanced nutrition is crucial for overall wellbeing. Eating a diet rich in fruits, vegetables, lean proteins, and whole grains can ensure the intake of essential nutrients. On the other hand, consuming excessive amounts of unhealthy foods such as processed snacks and fast food can lead to obesity, diabetes, and heart disease.

Moreover, understanding portion sizes is also crucial in maintaining a healthy diet. For instance, a standard serving of protein should be about the size of your palm, while a serving of vegetables and fruits can fill half your 

In [27]:
results_cot_adv = []
results_cot_adv_llama = cot_advanced(results_cot_adv, TOPICS, READABILITY_LEVELS, OUTPUT_LENGTHS, FLESCH_KINCAID_GRADE_TARGET, model_llama)
results_cot_adv_llama_df = pd.DataFrame(results_cot_adv_llama)
results_cot_adv_llama_df.to_csv("llama_readability_results_cot_advanced.csv", index=False)

Chain-of-Thought:   0%|                                     | 0/5 [00:00<?, ?it/s]

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31


Chain-of-Thought:  20%|█████                    | 1/5 [27:42<1:50:48, 1662.04s/it]

32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63


Chain-of-Thought:  40%|██████████               | 2/5 [49:59<1:13:33, 1471.21s/it]

...

96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127


Chain-of-Thought:  80%|████████████████████     | 4/5 [1:32:18<22:25, 1345.78s/it]

128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159


Chain-of-Thought: 100%|█████████████████████████| 5/5 [1:51:15<00:00, 1335.18s/it]

160
